In [2]:
#!/usr/bin/env python3

import os

from pslpython.model import Model
from pslpython.partition import Partition
from pslpython.predicate import Predicate
from pslpython.rule import Rule

MODEL_NAME = 'Same_Player_PSL'
DATA_DIR = os.path.join('PSL_data')

ADDITIONAL_PSL_OPTIONS = {
    'log4j.threshold': 'INFO'
}

ADDITIONAL_CLI_OPTIONS = [
    # '--postgres'
]

def main():
    model = Model(MODEL_NAME)

    # Add Predicates
    add_predicates(model)

    # Add Rules
    add_rules(model)

    # Inference
    results = infer(model)

    write_results(results, model)

# def main():
#     model = Model(MODEL_NAME)

#     # Add Predicates
#     add_predicates(model)

#     # Add Rules
#     add_rules(model)

#     # Weight Learning
#     learn(model)

#     print('Learned Rules:')
#     for rule in model.get_rules():
#         print('   ' + str(rule))

#     # Inference
#     results = infer(model)

#     write_results(results, model)


def write_results(results, model):
    out_dir = 'PSL_result'
    os.makedirs(out_dir, exist_ok = True)

    for predicate in model.get_predicates().values():
        if (predicate.closed()):
            continue

        out_path = os.path.join(out_dir, "%s.txt" % (predicate.name()))
        results[predicate].to_csv(out_path, sep = "\t", header = False, index = False)

def add_predicates(model):
    predicate = Predicate('playerBlock', closed = True, arg_types = [Predicate.ArgType.UNIQUE_STRING_ID, Predicate.ArgType.UNIQUE_STRING_ID])
    model.add_predicate(predicate)
    
    predicate = Predicate('playerName', closed = True, arg_types = [Predicate.ArgType.UNIQUE_STRING_ID, Predicate.ArgType.UNIQUE_STRING_ID])
    model.add_predicate(predicate)

    predicate = Predicate('samePlayer', closed = False, arg_types = [Predicate.ArgType.UNIQUE_STRING_ID, Predicate.ArgType.UNIQUE_STRING_ID])
    model.add_predicate(predicate)

    predicate = Predicate('simName', closed = True, arg_types = [Predicate.ArgType.UNIQUE_STRING_ID, Predicate.ArgType.UNIQUE_STRING_ID])
    model.add_predicate(predicate)

    
    
#     predicate = Predicate('Knows', closed = False, size = 2)
#     model.add_predicate(predicate)

#     predicate = Predicate('Likes', closed = True, size = 2)
#     model.add_predicate(predicate)

#     predicate = Predicate('Lived', closed = True, size = 2)
#     model.add_predicate(predicate)

# def add_rules(model):
#     model.add_rule(Rule('20: Lived(P1, L) & Lived(P2, L) & (P1 != P2) -> Knows(P1, P2) ^2'))
#     model.add_rule(Rule('5: Lived(P1, L1) & Lived(P2, L2) & (P1 != P2) & (L1 != L2) -> !Knows(P1, P2) ^2'))
#     model.add_rule(Rule('10: Likes(P1, L) & Likes(P2, L) & (P1 != P2) -> Knows(P1, P2) ^2'))
#     model.add_rule(Rule('5: Knows(P1, P2) & Knows(P2, P3) & (P1 != P3) -> Knows(P1, P3) ^2'))
#     model.add_rule(Rule('Knows(P1, P2) = Knows(P2, P1) .'))
#     model.add_rule(Rule('5: !Knows(P1, P2) ^2'))
    
    
def add_rules(model):
    # Symmetry
    model.add_rule(Rule('samePlayer(P1, P2) = samePlayer(P2, P1) .'))

    model.add_rule(Rule('40.0: playerBlock(P1, L) & playerBlock(P2, L) & (P1 != P2) -> samePlayer(P1, P2) ^2'))
    
    # Paper rectangle closure.
    model.add_rule(Rule("""
        40.0: playerBlock(A1, B1) & playerBlock(A2, B1)
            & simName(A1, A2)
            -> samePlayer(A1, A2) ^2
    """))

    
    
    # Pure transitivity
    model.add_rule(Rule("""
        40.0: playerBlock(A1, B) & playerBlock(A2, B) & playerBlock(A3, B)
            & samePlayer(A1, A2) & samePlayer(A2, A3)
            & (A1 != A3) & (A1 != A2) & (A2 != A3)
            -> samePlayer(A1, A3) ^2
    """))
    
#     # Self-refernece.
    model.add_rule(Rule("samePlayer(A, A) = 1.0 ."))

    # Negative priors.
    model.add_rule(Rule("1.0: !samePlayer(A1, A2) ^2"))

def add_learn_data(model):
    add_data('', model)

def add_eval_data(model):
    add_data('', model)   


def add_data(split, model):
    split_data_dir = os.path.join(DATA_DIR, split)
    for predicate in model.get_predicates().values():
        predicate.clear_data()

    path = os.path.join(DATA_DIR, 'birthBlock_obs.txt')
    model.get_predicate('playerBlock').add_data_file(Partition.OBSERVATIONS, path)

#     path = os.path.join(DATA_DIR, 'playerName_obs.txt')
#     model.get_predicate('playerName').add_data_file(Partition.OBSERVATIONS, path)

    path = os.path.join(DATA_DIR, 'simName_obs.txt')
    model.get_predicate('simName').add_data_file(Partition.OBSERVATIONS, path)

    path = os.path.join(DATA_DIR, 'same_player_target.txt')
    model.get_predicate('samePlayer').add_data_file(Partition.TARGETS, path)

#     path = os.path.join(DATA_DIR, 'knows_truth.txt')
#     model.get_predicate('Knows').add_data_file(Partition.TRUTH, path)
# def learn(model):
#     add_learn_data(model)
#     model.learn(additional_cli_optons = ADDITIONAL_CLI_OPTIONS, psl_config = ADDITIONAL_PSL_OPTIONS)
    
def infer(model):
    add_data('', model)
    return model.infer(additional_cli_optons = ADDITIONAL_CLI_OPTIONS, psl_config = ADDITIONAL_PSL_OPTIONS)

if (__name__ == '__main__'):
    main()


13028 [pslpython.model PSL] INFO --- 0    [main] INFO  org.linqs.psl.cli.Launcher  - Running PSL CLI Version 2.2.2-5f9a472
13488 [pslpython.model PSL] INFO --- 463  [main] INFO  org.linqs.psl.cli.Launcher  - Loading data
14812 [pslpython.model PSL] INFO --- 1787 [main] INFO  org.linqs.psl.cli.Launcher  - Data loading complete
14813 [pslpython.model PSL] INFO --- 1787 [main] INFO  org.linqs.psl.cli.Launcher  - Loading model from /var/folders/27/fcv785895zb4wczv3dzgn3gh0000gn/T/psl-python/Same_Player_PSL/Same_Player_PSL.psl
14972 [pslpython.model PSL] INFO --- 1947 [main] INFO  org.linqs.psl.cli.Launcher  - Model loading complete
14974 [pslpython.model PSL] INFO --- 1947 [main] INFO  org.linqs.psl.cli.Launcher  - Starting inference with class: org.linqs.psl.application.inference.MPEInference
16452 [pslpython.model PSL] INFO --- 3427 [main] INFO  org.linqs.psl.application.inference.MPEInference  - Grounding out model.
18356 [pslpython.model PSL] INFO --- 5331 [main] INFO  org.linqs.psl.ap